In [1]:
import sys
print("Python version:", sys.version)

Python version: 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]


In [3]:
#import matplotlib
#matplotlib.__version__

Wczytaj dane `Telco_Customer_Churn.csv` (zmienna objaśniana: Churn). Dopasuj jak najlepszy model KNN. Wykorzystaj  techniki feature engineeringu (skalowanie zmiennych, metryki odległości, eliminacja zmiennych) w celu zbadania wszystkich możliwości. 

In [4]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
#import matplotlib.pyplot as plt

#plt.style.use('seaborn-ticks')
#%matplotlib inline

churn = pd.read_csv('Telco_Customer_Churn.csv', na_values = ' ')
# churn.drop(columns = 'customerID', inplace = True)
churn.set_index('customerID', inplace = True)
churn.dropna(inplace = True)
churn

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No


In [5]:
churn.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [6]:
churn.shape

(7032, 20)

In [7]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   object 
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   object 
 3   Dependents        7032 non-null   object 
 4   tenure            7032 non-null   int64  
 5   PhoneService      7032 non-null   object 
 6   MultipleLines     7032 non-null   object 
 7   InternetService   7032 non-null   object 
 8   OnlineSecurity    7032 non-null   object 
 9   OnlineBackup      7032 non-null   object 
 10  DeviceProtection  7032 non-null   object 
 11  TechSupport       7032 non-null   object 
 12  StreamingTV       7032 non-null   object 
 13  StreamingMovies   7032 non-null   object 
 14  Contract          7032 non-null   object 
 15  PaperlessBilling  7032 non-null   object 
 16  PaymentMethod     7032 non-null 

In [8]:
churn.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.162400,32.421786,64.798208,2283.300441
std,0.368844,24.545260,30.085974,2266.771362
min,0.000000,1.000000,18.250000,18.800000
25%,0.000000,9.000000,35.587500,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.862500,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [9]:
target = 'Churn'
numFeatures = ['tenure',  'MonthlyCharges', 'TotalCharges']
catFeatures = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',  'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection','TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

In [10]:
# One hot encoding
dummLev = pd.get_dummies(churn[catFeatures], drop_first=True)

# Skalowanie
churn[numFeatures] = churn[numFeatures].apply(lambda x: (x-x.mean())/x.std())

# Połączenie zmiennych numerycznych, kategorycznych w postaci on hot encoding oraz targetu
df = pd.concat([churn[numFeatures], dummLev, churn[[target]]], axis=1)
df

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
customerID,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,-1.280157,-1.161611,-0.994123,0,False,True,False,False,True,False,...,False,False,False,False,False,True,False,True,False,No
5575-GNVDE,0.064298,-0.260859,-0.173727,0,True,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,No
3668-QPYBK,-1.239416,-0.363897,-0.959581,0,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,True,Yes
7795-CFOCW,0.512450,-0.747797,-0.195234,0,True,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,No
9237-HQITU,-1.239416,0.196164,-0.940391,0,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,-0.343113,0.664821,-0.129171,0,True,True,True,True,False,True,...,True,False,True,True,False,True,False,False,True,No
2234-XADUH,1.612459,1.276402,2.240896,0,False,True,True,True,False,True,...,True,False,True,True,False,True,True,False,False,No
4801-JZAZL,-0.872746,-1.169921,-0.854453,0,False,True,True,False,True,False,...,False,False,False,False,False,True,False,True,False,No


In [11]:
# lista zmiennych
features = df.columns.tolist()
features.remove(target)
features

['tenure',
 'MonthlyCharges',
 'TotalCharges',
 'SeniorCitizen',
 'gender_Male',
 'Partner_Yes',
 'Dependents_Yes',
 'PhoneService_Yes',
 'MultipleLines_No phone service',
 'MultipleLines_Yes',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No internet service',
 'OnlineSecurity_Yes',
 'OnlineBackup_No internet service',
 'OnlineBackup_Yes',
 'DeviceProtection_No internet service',
 'DeviceProtection_Yes',
 'TechSupport_No internet service',
 'TechSupport_Yes',
 'StreamingTV_No internet service',
 'StreamingTV_Yes',
 'StreamingMovies_No internet service',
 'StreamingMovies_Yes',
 'Contract_One year',
 'Contract_Two year',
 'PaperlessBilling_Yes',
 'PaymentMethod_Credit card (automatic)',
 'PaymentMethod_Electronic check',
 'PaymentMethod_Mailed check']

In [12]:
# Wczytanie potrzebnych bibliotek
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import neighbors
from sklearn.metrics import roc_auc_score

# Przygotujmy wrapper - na końcu pojawiają się *args i **kwargs - listy parametrów podawane w postaci słownika
# lub listy.
def CVTestKNN(nFolds = 5, randomState=2020, debug=False, features=features, *args, **kwargs):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # Listy na wyniki:
    testResults = []
    trainResults = []
    predictions = []
    indices = []
    
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = neighbors.KNeighborsClassifier(*args, **kwargs)
        # Wyświetlenie wywołania funkcji
        if debug:
            print(clf)
        # Trenowanie modelu
        clf.fit(df[features].iloc[train], df[target].iloc[train])

        # Przygotowanie prognoz dla zbioru traeningowego i testowego
        # Sklearn zwracał będzie dwie kolumny prawdopodobieństw dla obydwu klas
        predsTrain = clf.predict_proba(df.iloc[train][features])[:,1]
        preds = clf.predict_proba(df.iloc[test][features])[:,1]
        
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        
        # Policzenie statystyka lda każdego foldu
        trainScore = roc_auc_score((df[target].iloc[train]=="Yes").astype(int), predsTrain)
        testScore = roc_auc_score((df[target].iloc[test]=="Yes").astype(int), preds)
        
        # Zapisanie wyników do list
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)
        
    return trainResults, testResults, predictions, indices

In [13]:
trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=50, n_jobs=-1, p=2, debug=True)
print(np.mean(testResults))

KNeighborsClassifier(n_jobs=-1, n_neighbors=50)


  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Train AUC: 0.8538812484596053 Valid AUC: 0.8254525250361485
KNeighborsClassifier(n_jobs=-1, n_neighbors=50)
Train AUC: 0.845752609266083 Valid AUC: 0.851786909650411
KNeighborsClassifier(n_jobs=-1, n_neighbors=50)
Train AUC: 0.8523275653229172 Valid AUC: 0.8248438011289266
KNeighborsClassifier(n_jobs=-1, n_neighbors=50)
Train AUC: 0.8522424746130023 Valid AUC: 0.8316225254146603
KNeighborsClassifier(n_jobs=-1, n_neighbors=50)
Train AUC: 0.8463230396528048 Valid AUC: 0.8539961013645224
0.8375403725189339


In [14]:
# Ta pętla przechodzi po wartościach k i dla każdej wykonuje walidację krzyżową.
for k in [1, 3, 5, 10, 15, 30, 50, 75, 100, 150]:
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=k, n_jobs=-1, p=2)
    print(k, np.mean(trainResults), np.mean(testResults))

1 0.9967475668201242 0.6613431727357375
3 0.9235603147606766 0.7512697804474266
5 0.898475513807886 0.7856078917808684
10 0.8763273830124418 0.8203497410842349
15 0.8676330663413694 0.8273169043513396
30 0.8561997788815798 0.8362433048029214
50 0.8501053874628826 0.8375403725189339
75 0.8474293846224968 0.8390006738709654
100 0.8451869964833476 0.8392430073688297
150 0.8411830625788641 0.837078986143216


In [15]:
# Ta pętla przechodzi po wartościach k i dla każdej wykonuje walidację krzyżową.
for k in [75, 85, 95, 100, 110]:
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=k, n_jobs=-1, p=2)
    print(k, np.mean(trainResults), np.mean(testResults))

75 0.8474293846224968 0.8390006738709654
85 0.8467123374332353 0.8391836528673087
95 0.8456399999229228 0.8393230494256734
100 0.8451869964833476 0.8392430073688297
110 0.8439927123808136 0.8387517999526539


In [16]:
# Ta pętla przechodzi po wartościach k i dla każdej wykonuje walidację krzyżową.
for k in [5, 10, 15, 30, 50, 75, 100, 150]:
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=k, n_jobs=-1, p=1)
    print(k, np.mean(trainResults), np.mean(testResults))

5 0.9008565069781118 0.7904979063627375
10 0.8762995089423431 0.8187806287320145
15 0.8682384165246612 0.8271379486796908
30 0.8560881228320911 0.8360496263427623
50 0.8502677320263411 0.8376624304215368
75 0.8483633117943263 0.8402625406367037
100 0.8463164899026545 0.8401238998940407
150 0.842401678286676 0.838606203831195


In [17]:
# Ta pętla przechodzi po wartościach k i dla każdej wykonuje walidację krzyżową.
for k in [75, 85, 90, 95, 100, 110]:
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=k, n_jobs=-1, p=1)
    print(k, np.mean(trainResults), np.mean(testResults))

75 0.8483633117943263 0.8402625406367037
85 0.8479966475651797 0.8404496583556463
90 0.8474186691438093 0.8404202843857362
95 0.846857543661045 0.8401708875325733
100 0.8463164899026545 0.8401238998940407
110 0.8455158116883933 0.8398186016484953


In [18]:
# Dodajemy każdą zmienną jako osobnego kandydata.
results = []
for feature in features:
    # Estymacja modelu
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=[feature])
    print('feature: ', feature, ', ', 'mean: ', np.mean(testResults))
    # Zapisanie wyniku jako do obiektu results
    results.append((feature, np.mean(testResults)))

feature:  tenure ,  mean:  0.7326603465014033
feature:  MonthlyCharges ,  mean:  0.6754199267714647
feature:  TotalCharges ,  mean:  0.6491756896172424
feature:  SeniorCitizen ,  mean:  0.5627985282713739
feature:  gender_Male ,  mean:  0.5046297200293702
feature:  Partner_Yes ,  mean:  0.5847542116267869
feature:  Dependents_Yes ,  mean:  0.5840008333378223
feature:  PhoneService_Yes ,  mean:  0.4953839300241844
feature:  MultipleLines_No phone service ,  mean:  0.4953668085688607
feature:  MultipleLines_Yes ,  mean:  0.5025870762565734
feature:  InternetService_Fiber optic ,  mean:  0.6728915566719593
feature:  InternetService_No ,  mean:  0.6061784904356398
feature:  OnlineSecurity_No internet service ,  mean:  0.6061784904356398
feature:  OnlineSecurity_Yes ,  mean:  0.5875279203073437
feature:  OnlineBackup_No internet service ,  mean:  0.6061784904356398
feature:  OnlineBackup_Yes ,  mean:  0.5439196039694988
feature:  DeviceProtection_No internet service ,  mean:  0.606178490435

In [19]:
results = sorted(results, key=lambda x: x[1], reverse=True)
results

[('tenure', 0.7326603465014033),
 ('MonthlyCharges', 0.6754199267714647),
 ('InternetService_Fiber optic', 0.6728915566719593),
 ('PaymentMethod_Electronic check', 0.6616015924921236),
 ('TotalCharges', 0.6491756896172424),
 ('Contract_Two year', 0.6456642377079682),
 ('PaperlessBilling_Yes', 0.6065412554387012),
 ('InternetService_No', 0.6061784904356398),
 ('OnlineSecurity_No internet service', 0.6061784904356398),
 ('OnlineBackup_No internet service', 0.6061784904356398),
 ('DeviceProtection_No internet service', 0.6061784904356398),
 ('TechSupport_No internet service', 0.6061784904356398),
 ('StreamingTV_No internet service', 0.6061784904356398),
 ('StreamingMovies_No internet service', 0.6061784904356398),
 ('OnlineSecurity_Yes', 0.5875279203073437),
 ('Partner_Yes', 0.5847542116267869),
 ('TechSupport_Yes', 0.5846196752217294),
 ('Dependents_Yes', 0.5840008333378223),
 ('Contract_One year', 0.5820378095797688),
 ('SeniorCitizen', 0.5627985282713739),
 ('PaymentMethod_Credit card 

In [20]:
featuresSet2 = [x for x,y in results if y > 0.5]
print(featuresSet2)

['tenure', 'MonthlyCharges', 'InternetService_Fiber optic', 'PaymentMethod_Electronic check', 'TotalCharges', 'Contract_Two year', 'PaperlessBilling_Yes', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineBackup_No internet service', 'DeviceProtection_No internet service', 'TechSupport_No internet service', 'StreamingTV_No internet service', 'StreamingMovies_No internet service', 'OnlineSecurity_Yes', 'Partner_Yes', 'TechSupport_Yes', 'Dependents_Yes', 'Contract_One year', 'SeniorCitizen', 'PaymentMethod_Credit card (automatic)', 'OnlineBackup_Yes', 'PaymentMethod_Mailed check', 'StreamingMovies_Yes', 'DeviceProtection_Yes', 'gender_Male', 'MultipleLines_Yes']


In [21]:
trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet2)
print('mean(testResults): ', np.mean(testResults))

mean(testResults):  0.8402977181573681


In [22]:
featuresSet4 = [x for x,y in results if y > 0.49538]
print(featuresSet4)
trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet4)
print('mean(testResults): ', np.mean(testResults))

['tenure', 'MonthlyCharges', 'InternetService_Fiber optic', 'PaymentMethod_Electronic check', 'TotalCharges', 'Contract_Two year', 'PaperlessBilling_Yes', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineBackup_No internet service', 'DeviceProtection_No internet service', 'TechSupport_No internet service', 'StreamingTV_No internet service', 'StreamingMovies_No internet service', 'OnlineSecurity_Yes', 'Partner_Yes', 'TechSupport_Yes', 'Dependents_Yes', 'Contract_One year', 'SeniorCitizen', 'PaymentMethod_Credit card (automatic)', 'OnlineBackup_Yes', 'PaymentMethod_Mailed check', 'StreamingMovies_Yes', 'DeviceProtection_Yes', 'gender_Male', 'MultipleLines_Yes', 'StreamingTV_Yes', 'PhoneService_Yes']
mean(testResults):  0.8406823253666559


In [23]:
featuresSet3 = [x for x,y in results if y > 0.4954]
print(featuresSet3)
trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet3)
print('mean(testResults): ', np.mean(testResults))

['tenure', 'MonthlyCharges', 'InternetService_Fiber optic', 'PaymentMethod_Electronic check', 'TotalCharges', 'Contract_Two year', 'PaperlessBilling_Yes', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineBackup_No internet service', 'DeviceProtection_No internet service', 'TechSupport_No internet service', 'StreamingTV_No internet service', 'StreamingMovies_No internet service', 'OnlineSecurity_Yes', 'Partner_Yes', 'TechSupport_Yes', 'Dependents_Yes', 'Contract_One year', 'SeniorCitizen', 'PaymentMethod_Credit card (automatic)', 'OnlineBackup_Yes', 'PaymentMethod_Mailed check', 'StreamingMovies_Yes', 'DeviceProtection_Yes', 'gender_Male', 'MultipleLines_Yes', 'StreamingTV_Yes']
mean(testResults):  0.8408354494318407


In [24]:
# Przygotujmy kopię zapasową zbioru.
dfCopy = df.copy()

In [25]:
# Dodajemy każdą zmienną jako kandydata.
results2 = []
for feature in featuresSet3:
    df = dfCopy.copy()
    # Dwukrotnie zwiększamy zmienną
    df[feature]=df[feature]*2
#     W walidujemy model
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet3)
    print('feature: ', feature, ', ', 'mean: ', np.mean(testResults))
    results2.append((feature, np.mean(testResults)))

feature:  tenure ,  mean:  0.8412325609329676
feature:  MonthlyCharges ,  mean:  0.8400990080212901
feature:  InternetService_Fiber optic ,  mean:  0.8403546829027846
feature:  PaymentMethod_Electronic check ,  mean:  0.8391278241082984
feature:  TotalCharges ,  mean:  0.841164876127127
feature:  Contract_Two year ,  mean:  0.8410243121511355
feature:  PaperlessBilling_Yes ,  mean:  0.8405114424725679
feature:  InternetService_No ,  mean:  0.8408395260149215
feature:  OnlineSecurity_No internet service ,  mean:  0.8408395260149215
feature:  OnlineBackup_No internet service ,  mean:  0.8408395260149215
feature:  DeviceProtection_No internet service ,  mean:  0.8408395260149215
feature:  TechSupport_No internet service ,  mean:  0.8408395260149215
feature:  StreamingTV_No internet service ,  mean:  0.8408395260149215
feature:  StreamingMovies_No internet service ,  mean:  0.8408395260149215
feature:  OnlineSecurity_Yes ,  mean:  0.8395436757400608
feature:  Partner_Yes ,  mean:  0.839056

In [26]:
for k in [1.5, 2, 3, 4, 5]:
    df = dfCopy.copy()
    # k krotnie zwiększamy zmienną
    df["tenure"]=df["tenure"]*k
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet3)
    print(k, np.mean(testResults))

1.5 0.8413220979498807
2 0.8412325609329676
3 0.8410717110706658
4 0.8409857098587349
5 0.8402864637410445


In [27]:
for k in [1.5, 2, 3, 4, 5]:
    df = dfCopy.copy()
    # k krotnie zwiększamy zmienną
    df["tenure"]=df["tenure"]*1.5
    df["TotalCharges"]=df["TotalCharges"]*k
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet3)
    print(k, np.mean(testResults))

1.5 0.841103859938644
2 0.8412088141785924
3 0.8409733836123436
4 0.8412597980079157
5 0.8416569777486103


In [28]:
for k in [1.5, 2, 3, 4, 5]:
    df = dfCopy.copy()
    # k krotnie zwiększamy zmienną
    df["tenure"]=df["tenure"]*1.5
    df["TotalCharges"]=df["TotalCharges"]*5
    df["Contract_Two year"]=df["Contract_Two year"]*k
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet3)
    print(k, np.mean(testResults))

1.5 0.8422033840676635
2 0.8425032094401311
3 0.8428860537998328
4 0.8432147492780878
5 0.8433430502321372


In [31]:
wyniki = [] # do wykresu, ale musi działac matplotlib

for k in [1.5, 2, 3, 4, 5]:
    df = dfCopy.copy()
    # k krotnie zwiększamy zmienną
    df["tenure"]=df["tenure"]*1.5
    df["TotalCharges"]=df["TotalCharges"]*5
    df["Contract_Two year"]=df["Contract_Two year"]*k
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=90, n_jobs=-1, p=1, features=featuresSet3)
    wyniki.append((k, np.mean(testResults)))
    print(k, np.mean(testResults))



1.5 0.8422033840676635
2 0.8425032094401311
3 0.8428860537998328
4 0.8432147492780878
5 0.8433430502321372
